# Cosmos DB

Cosmos DBを扱う。

In [1]:
from azure.cosmos import CosmosClient, PartitionKey
from dotenv import load_dotenv
import os
import urllib3
import hashlib

In [2]:
load_dotenv()

cosmos_endpoint = os.environ["COSMOS_ENDPOINT"]
#エミュレーター使ってる場合はSSLまわりが、まあ、なんというか、面倒なので検証しない
connection_verify = not "localhost" in cosmos_endpoint
if not connection_verify:
    urllib3.disable_warnings()

クライアントとデータベース、コンテナーを準備する。

In [3]:
client = CosmosClient(
    url=cosmos_endpoint,
    credential=os.environ["COSMOS_KEY"],
    connection_verify=connection_verify,
)

database = client.create_database_if_not_exists(
    id="example_database",
    offer_throughput=400,
)

container = database.create_container_if_not_exists(
    id="example_container",
    partition_key=PartitionKey(
        path="/data_group",
    ),
)

サンプルデータを投入する。
項目は次の通り。

- `id`：連番
- `data_group`：パーティションキーなので、適当にバラけさせるため`id`を12で割った余り
- `data`：`id`のSHA-1ハッシュ値

In [4]:
for i in range(0, 200):
    hash_data = hashlib.sha1(str(i).encode())
    item = {
        "id": str(i),
        "data_group": i%12,
        "data": hash_data.hexdigest(),
    }
    container.upsert_item(item)

データを1件読み取る。

In [5]:
item = container.read_item("13", 13%12)
item["id"], item["data_group"], item["data"]

('13', 1, 'bd307a3ec329e10a2cff8fb87480823da114f8f4')

全件読み取る。
Webのコンソールだと一度の問い合わせで最大100件までしか取得できないけれど、SDK使えばその縛りはなさそう。

In [6]:
len([item for item in container.read_all_items()])

200

クエリーを指定して全件取得する。
パーティションキーを指定しないクエリーの場合は`enable_cross_partition_query=True`を設定しなければいけない。
内部的には次のようなリクエストヘッダーを設定してHTTPリクエストを送信しているみたい。

```
x-ms-documentdb-query-enablecrosspartition: true
```

In [7]:
len([item for item in container.query_items("select * from c", enable_cross_partition_query=True)])

200

条件式を書いたりパラメーター渡したりしてみる。

In [8]:
[
    item
    for item in container.query_items(
        query=(
            "select c.id, c.data_group, c.data from c"
            " where c.data_group = @data_group"
            " offset 0 limit 3"
        ),
        parameters=[
            dict(name="@data_group", value=1)
        ],
    )
]

[{'id': '1',
  'data_group': 1,
  'data': '356a192b7913b04c54574d18c28d46e6395428ab'},
 {'id': '13',
  'data_group': 1,
  'data': 'bd307a3ec329e10a2cff8fb87480823da114f8f4'},
 {'id': '25',
  'data_group': 1,
  'data': 'f6e1126cedebf23e1463aee73f9df08783640400'}]